In [75]:
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup

In [76]:
website_text = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_text,'xml')

In [77]:
table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')

In [78]:
data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

In [101]:
df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df.drop([0],axis=0,inplace=True)
df.reset_index()
df.head()

,PostalCode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


In [110]:
df['Borough'].replace('Not assigned', np.nan, inplace=True)
df.dropna(subset=['Borough'], inplace=True)

In [111]:
df.head()

,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


In [112]:
df1=df.groupby("PostalCode").agg(lambda x:','.join(set(x)))

In [113]:
df1.loc[df1['Neighbourhood']=='Not assigned','Neighbourhood']=df1.loc[df1['Neighbourhood']=='Not assigned','Borough']

In [114]:
df1.head()

,Borough,Neighbourhood
PostalCode,,
M1B,Scarborough,"Malvern,Rouge"
M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek"
M1E,Scarborough,"Morningside,Guildwood,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [115]:
df1.shape

(103, 2)

Step 2 - Add Geospatial Data

we need to get the latitude and the longitude coordinates of each neighborhood.

In [126]:
#Get data lat/long data from csv
lat_long_coord_df = pd.read_csv(r"C:\Users\Luis\Documents\Geospatial_Coordinates.csv")

#rename columns and set the index to be Postcode

lat_long_coord_df.columns = ["Postcode", "Latitude", "Longitude"]
if(lat_long_coord_df.index.name != 'Postcode'):
    lat_long_coord_df = lat_long_coord_df.set_index('Postcode')
    
lat_long_coord_df.head()

,Latitude,Longitude
Postcode,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [127]:
df2 = df1.join(lat_long_coord_df)
df2.head(10)

,Borough,Neighbourhood,Latitude,Longitude
PostalCode,,,,
M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
M1C,Scarborough,"Port Union,Rouge Hill,Highland Creek",43.784535,-79.160497
M1E,Scarborough,"Morningside,Guildwood,West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476
M1J,Scarborough,Scarborough Village,43.744734,-79.239476
M1K,Scarborough,"Ionview,East Birchmount Park,Kennedy Park",43.727929,-79.262029
M1L,Scarborough,"Golden Mile,Oakridge,Clairlea",43.711112,-79.284577
M1M,Scarborough,"Cliffcrest,Scarborough Village West,Cliffside",43.716316,-79.239476
